# [185. Department Top Three Salaries](https://leetcode.com/problems/department-top-three-salaries/?envType=study-plan-v2&envId=top-sql-50)

Table: Employee

<pre>+--------------+---------+
| Column Name  | Type    |
+--------------+---------+
| id           | int     |
| name         | varchar |
| salary       | int     |
| departmentId | int     |
+--------------+---------+</pre>
id is the primary key (column with unique values) for this table.
departmentId is a foreign key (reference column) of the ID from the Department table.
Each row of this table indicates the ID, name, and salary of an employee. It also contains the ID of their department.


Table: Department

<pre>+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| id          | int     |
| name        | varchar |
+-------------+---------+</pre>
id is the primary key (column with unique values) for this table.
Each row of this table indicates the ID of a department and its name.


A company's executives are interested in seeing who earns the most money in each of the company's departments. A high earner in a department is an employee who has a salary in the top three unique salaries for that department.

Write a solution to find the employees who are high earners in each of the departments.

Return the result table in any order.

The result format is in the following example.



Example 1:

Input:
Employee table:
<pre>+----+-------+--------+--------------+
| id | name  | salary | departmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 85000  | 1            |
| 2  | Henry | 80000  | 2            |
| 3  | Sam   | 60000  | 2            |
| 4  | Max   | 90000  | 1            |
| 5  | Janet | 69000  | 1            |
| 6  | Randy | 85000  | 1            |
| 7  | Will  | 70000  | 1            |
+----+-------+--------+--------------+</pre>
Department table:
<pre>+----+-------+
| id | name  |
+----+-------+
| 1  | IT    |
| 2  | Sales |
+----+-------+</pre>
Output:
<pre>+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Max      | 90000  |
| IT         | Joe      | 85000  |
| IT         | Randy    | 85000  |
| IT         | Will     | 70000  |
| Sales      | Henry    | 80000  |
| Sales      | Sam      | 60000  |
+------------+----------+--------+</pre>
Explanation:
In the IT department:
- Max earns the highest unique salary
- Both Randy and Joe earn the second-highest unique salary
- Will earns the third-highest unique salary

In the Sales department:
- Henry earns the highest salary
- Sam earns the second-highest salary
- There is no third-highest salary as there are only two employees

In [1]:
#Pandas schema

import pandas as pd

data = [[1, 'Joe', 85000, 1], [2, 'Henry', 80000, 2], [3, 'Sam', 60000, 2], [4, 'Max', 90000, 1],
        [5, 'Janet', 69000, 1], [6, 'Randy', 85000, 1], [7, 'Will', 70000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype(
    {'id': 'Int64', 'name': 'object', 'salary': 'Int64', 'departmentId': 'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id': 'Int64', 'name': 'object'})

# pandas df to spark df

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.getOrCreate()

employee_df = spark.createDataFrame(employee)
employee_df.show(truncate=False)

department_df = spark.createDataFrame(department)
department_df.show(truncate=False)

+---+-----+------+------------+
|id |name |salary|departmentId|
+---+-----+------+------------+
|1  |Joe  |85000 |1           |
|2  |Henry|80000 |2           |
|3  |Sam  |60000 |2           |
|4  |Max  |90000 |1           |
|5  |Janet|69000 |1           |
|6  |Randy|85000 |1           |
|7  |Will |70000 |1           |
+---+-----+------+------------+

+---+-----+
|id |name |
+---+-----+
|1  |IT   |
|2  |Sales|
+---+-----+



In [7]:
from pyspark.sql import Window
#  solving in Spark Dataframe
spec = Window.partitionBy('d.name').orderBy(col('salary').desc())

department_df.alias('d')\
        .join(employee_df.alias('e'), department_df.id==employee_df.departmentId,'left')\
        .withColumn('rnk',dense_rank().over(spec))\
        .where('rnk<4')\
        .select(col('d.name').alias('Department'),col('e.name').alias('Employee'),col('salary'))\
        .show()


+----------+--------+------+
|Department|Employee|salary|
+----------+--------+------+
|        IT|     Max| 90000|
|        IT|   Randy| 85000|
|        IT|     Joe| 85000|
|        IT|    Will| 70000|
|     Sales|   Henry| 80000|
|     Sales|     Sam| 60000|
+----------+--------+------+



In [ ]:
# in Spark SQL
employee_df.createOrReplaceTempView('employee')
department_df.createOrReplaceTempView('department')

spark.sql('''
with cte as(
select d.name as Department, e.name as Employee, Salary,
     dense_rank() over(partition by d.name order by salary desc) as rnk
from employee e
    left join Department d
    on d.id= e.DepartmentID
    )

select Department, Employee, Salary from cte
where rnk <4
''').show()

+----------+--------+------+
|Department|Employee|Salary|
+----------+--------+------+
|        IT|     Max| 90000|
|        IT|     Joe| 85000|
|        IT|   Randy| 85000|
|        IT|    Will| 70000|
|     Sales|   Henry| 80000|
|     Sales|     Sam| 60000|
+----------+--------+------+


In [ ]:
spark.stop()